In [8]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import time
import dask_cudf
import cudf
import cuml

#from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
#from sklearn.grid_search import GridSearchCV



In [4]:
%mkdir /rapids/notebooks/wip/notebooks-extended/data/nyctaxi2
%cd /rapids/notebooks/wip/notebooks-extended/data/nyctaxi2
!wget https://raw.githubusercontent.com/suvoooo/Machine_Learning/master/winequality-red.csv

mkdir: cannot create directory ‘/rapids/notebooks/wip/notebooks-extended/data/nyctaxi2’: File exists
/rapids/notebooks/wip/notebooks-extended/data/nyctaxi2
--2019-04-24 02:49:50--  https://raw.githubusercontent.com/suvoooo/Machine_Learning/master/winequality-red.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84199 (82K) [text/plain]
Saving to: ‘winequality-red.csv’

winequality-red.csv 100%[===================>]  82.23K  --.-KB/s    in 0.01s   

2019-04-24 02:49:50 (7.12 MB/s) - ‘winequality-red.csv’ saved [84199/84199]



In [11]:
startT = time.time()


winedf = cudf.read_csv('/rapids/notebooks/wip/notebooks-extended/data/nyctaxi2/winequality-red.csv',sep=';')

#print winedf.isnull().sum() check for missing data

print(winedf.head(3))

   fixed acidity    volatile acidity  citric acid      residual sugar            chlorides  free sulfur dioxide  total sulfur dioxide ...  quality
0            7.4  0.7000000000000001          0.0  1.9000000000000001  0.07600000000000001                 11.0                  34.0 ...        5
1            7.8                0.88          0.0                 2.6                0.098                 25.0                  67.0 ...        5
2            7.8  0.7600000000000001         0.04                 2.3                0.092                 15.0                  54.0 ...        5
[4 more columns]


In [14]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++
#+ check whether the labels are unblanced or not
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++

#print winedf.shape
#ylab = winedf[['quality']]
#print ylab.shape
#print winedf['quality'].value_counts() # indeed it is

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# check the correlation plot

#winecorr = winedf.corr()
#s=sns.heatmap(winecorr)
#s.set_yticklabels(s.get_yticklabels(),rotation=30,fontsize=7)
#s.set_xticklabels(s.get_xticklabels(),rotation=30,fontsize=7)

#plt.show() # as expected high correlation between acidity and pH

# individual correlation plot
#plt.subplot(1,2,1)
#plt.scatter(winedf['fixed acidity'], winedf['pH'], s=winedf['quality']*5, color='magenta', alpha=0.3)
#plt.xlabel('Fixed Acidity')
#plt.ylabel('pH')
#plt.subplot(1,2,2)
#plt.scatter(winedf['fixed acidity'], winedf['residual sugar'], s=winedf['quality']*5, color='purple', alpha=0.3)
#plt.xlabel('Fixed Acidity')
#plt.ylabel('Residual Sugar')
#plt.tight_layout()
#plt.show()

X=winedf.drop(['quality'])
Y=winedf['quality']

print(type(X), type(Y))
print(X.head(3))


<class 'cudf.dataframe.dataframe.DataFrame'> <class 'cudf.dataframe.series.Series'>
   fixed acidity    volatile acidity  citric acid      residual sugar            chlorides  free sulfur dioxide  total sulfur dioxide ...  alcohol
0            7.4  0.7000000000000001          0.0  1.9000000000000001  0.07600000000000001                 11.0                  34.0 ...      9.4
1            7.8                0.88          0.0                 2.6                0.098                 25.0                  67.0 ...      9.8
2            7.8  0.7600000000000001         0.04                 2.3                0.092                 15.0                  54.0 ...      9.8
[3 more columns]


In [ ]:
#++++++++++++++++++++++++++++++++
# create the pipeline object
#++++++++++++++++++++++++++++++++
steps = [('scaler', StandardScaler()), ('SVM', SVC())]
pipeline = Pipeline(steps)

In [ ]:
#++++++++++++++++++++++++++++++++++++++
#+ create the hyperparameter space
#++++++++++++++++++++++++++++++++++++++

parameteres = {'SVM__C':[0.001,0.1,10,100,10e5], 'SVM__gamma':[0.1,0.01]}

In [ ]:
#++++++++++++++++++++++++++++++++++++
#+ create train and test sets
#++++++++++++++++++++++++++++++++++++

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=30, stratify=Y)
#print X_test.shape

In [ ]:
#++++++++++++++++++++++++++++++
#+ Grid Search Cross Validation
#++++++++++++++++++++++++++++++
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=5)

grid.fit(X_train, y_train)

print "score = %3.2f" %(grid.score(X_test,y_test))

#pparam=pprint.PrettyPrinter(indent=2)

print grid.best_params_
endT = time.time()

print "total time elapsed = %3.3f"%(endT-startT)